# **Exercise 2: Estimate EPC counterfactuals under retrofit actions**

## Overview

In Exercise 1, we built descriptive models to understand which building characteristics are most associated with EPC performance. Here, we implement a simple *counterfactual (“what‑if”) analysis*:

1. Fit a model that approximates the EPC current energy efficiency score (`CURRENT_ENERGY_EFFICIENCY`) from a set of features.
2. Apply retrofit actions on a hold‑out set by changing EPC input variables (e.g., wall efficiency band), and predict the counterfactual EPC score.

> **Note**: This does *not* predict realised post‑retrofit bills. It estimates how the EPC score would change under modified building characteristics, using a data‑driven approximation of the EPC calculation logic.


## Set up

Load libraries and the EPC dataset. As in Exercise 1, we drop columns with missing values for simplicity (you can revisit this and use imputation later). The target (outcome) variable is `CURRENT_ENERGY_EFFICIENCY`. To avoid data leakage, be careful not to use any EPC output variables as predictors.

In [1]:
# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression

# Plot defaults
plt.rcParams['figure.constrained_layout.use'] = True
plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = (10, 3)

def accuracy_metrics(actual, predictions):
    ''' Estimate predictive accuracy metrics '''
    actual_copy = actual.copy().reshape(-1,1)
    predictions_copy = predictions.copy().reshape(-1,1)
    
    error = actual_copy - predictions_copy    
    assert(error.shape[0] == len(actual_copy))
    if error.ndim > 1:        
        assert(error.shape[1] == 1)

    mse = np.square(error).mean()
    rmse = np.sqrt( mse )
    mae = np.abs(error).mean()
    print(f'MSE: {mse}')
    print(f'RMSE: {rmse}')
    print(f'MAE: {mae}')
    return rmse, mae

In [2]:
# load data
# Update path to your local folder containing certificates.csv
path = r'C:\Users\ucbva19\Git projects\BENV0092\data raw\epc-certificates-Islington'
df = pd.read_csv(f"{path}\\certificates.csv")  # change path

print('(%) of missing data per column')
print((100*df.isna().sum()/len(df)).sort_values()[::-1])
display(df.head(3))

# For simplicity, drop columns with any NaNs (you can improve this later)
df = df.dropna(axis=1)
assert(df.isna().sum().sum() == 0)

(%) of missing data per column
LOW_ENERGY_FIXED_LIGHT_COUNT    100.0
SHEATING_ENERGY_EFF             100.0
SHEATING_ENV_EFF                100.0
CONSTITUENCY_LABEL              100.0
COUNTY                          100.0
                                ...  
HOT_WATER_ENV_EFF                 0.0
HOT_WATER_ENERGY_EFF              0.0
LOW_ENERGY_LIGHTING               0.0
MULTI_GLAZE_PROPORTION            0.0
LMK_KEY                           0.0
Length: 93, dtype: float64


,LMK_KEY,ADDRESS1,ADDRESS2,ADDRESS3,POSTCODE,BUILDING_REFERENCE_NUMBER,CURRENT_ENERGY_RATING,POTENTIAL_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,POTENTIAL_ENERGY_EFFICIENCY,...,CONSTITUENCY_LABEL,POSTTOWN,CONSTRUCTION_AGE_BAND,LODGEMENT_DATETIME,TENURE,FIXED_LIGHTING_OUTLETS_COUNT,LOW_ENERGY_FIXED_LIGHT_COUNT,UPRN,UPRN_SOURCE,REPORT_TYPE
0,05a4e13c8f8892b40f457e58195271839c253e738b2a4a...,Flat A,32 Hanley Road,NaN,N4 3DR,10008499045,D,C,68,77,...,NaN,LONDON,England and Wales: 1900-1929,2025-11-30 22:21:53,Rented (social),10,NaN,5.300040e+09,Energy Assessor,100
1,07c0c456133427557a94e545821381d14767982acbf6cb...,Flat 17,Park Gate House 250-252,Seven Sisters Road,N4 2HU,10003804896,B,B,82,82,...,NaN,LONDON,England and Wales: 2003-2006,2025-11-30 20:48:23,Owner-occupied,6,NaN,5.300104e+09,Energy Assessor,100
2,09ff1f899a2a4bfa4b183664b5d95962f6f636beba7616...,Flat 13,Park Gate House 250-252,Seven Sisters Road,N4 2HU,10007227906,B,B,84,84,...,NaN,LONDON,England and Wales: 2003-2006,2025-11-30 20:47:40,Owner-occupied,6,NaN,5.300104e+09,Energy Assessor,100


## Fit Predictive Models

First, we train two predictive models and select the best one. As in Exercise 1, features should be based on building characteristics (geometry, built form, efficiency bands for fabric, heating, etc.), whereas EPC outputs (costs, energy consumption, ratings) should *not* be used as features. 

In [3]:
target_variable = 'CURRENT_ENERGY_EFFICIENCY'

# further separate features into numerical, categorical, and ordinal
numerical_features = ['TOTAL_FLOOR_AREA', 'FIXED_LIGHTING_OUTLETS_COUNT', 'LOW_ENERGY_LIGHTING']
ordinal_features = ['WALLS_ENERGY_EFF', 'MAINHEAT_ENERGY_EFF', 'LIGHTING_ENERGY_EFF', 'HOT_WATER_ENERGY_EFF', 'WINDOWS_ENERGY_EFF']
categorical_features = ['PROPERTY_TYPE', 'BUILT_FORM']

feature_list = numerical_features + ordinal_features + categorical_features

Y = df[target_variable]
X = df[feature_list]

# Training/ test split
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.33, random_state=42)

# Preprocessing
categories_list = [df['PROPERTY_TYPE'].unique(), df['BUILT_FORM'].unique()]

ord_list =   [['Very Poor', 'Poor', 'Average', 'Good', 'Very Good'] for feat in ordinal_features]

# create a preprocessor that implements one-hot and ordinal encoding
preprocessor = ColumnTransformer(
    transformers=[("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
                  ("ord", OrdinalEncoder(categories = ord_list), ordinal_features),
                  ("num", "passthrough", numerical_features),])

We compare two models, namely, linear regression and extremely randomized trees (ExtraTrees). You are welcome to try other predictive models as well.

In [4]:
# Model 1: Linear Regression
lr_model = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("regressor", LinearRegression()),
    ]
)

# Model 2: ExtraTrees
et_model = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("regressor", ExtraTreesRegressor(
            n_estimators=500, random_state=42, n_jobs=-1))])

lr_model.fit(train_X, train_Y)
et_model.fit(train_X, train_Y)

pred_lr = lr_model.predict(test_X)
pred_et = et_model.predict(test_X)

print('Accuracy metrics LR')
rmse_lr, mae_lr = accuracy_metrics(test_Y.values, pred_lr)

print('Accuracy metrics ET')
rmse_et, mae_et = accuracy_metrics(test_Y.values, pred_et)

# Select the model with lower RMSE or MAE (your choice)
model = et_model if mae_et <= mae_lr else lr_model
print("Selected model:", type(model.named_steps['regressor']).__name__)

Accuracy metrics LR
MSE: 32.1814364402253
RMSE: 5.672868449049854
MAE: 4.284020942917015
Accuracy metrics ET
MSE: 34.231407692187005
RMSE: 5.850761291677093
MAE: 4.020642464296644
Selected model: ExtraTreesRegressor


## Define retrofit actions (monotonic upgrades)

We model retrofit actions by **upgrading efficiency bands** in a *monotonic* way.

- A dwelling is *eligible* for an action if it is **below** the target band.
- If the dwelling is already at or above the target, the action has **no effect**.

This keeps counterfactuals sane (e.g., we never “downgrade” a building).


In [5]:
# Helper: upgrade a single ordinal feature to at least a target level

EFF_ORDER = {"Very Poor": 1,
             "Poor": 2,
             "Average": 3,
             "Good": 4,
             "Very Good": 5}

def upgrade_to_level(df_in, feature, target_level, order = EFF_ORDER):
    df_out = df_in.copy()
    idx_map = {v: i for i, v in enumerate(order)}

    current = df_out[feature].astype(str)
    current_idx = current.map(idx_map)
    target_idx = idx_map[target_level]

    eligible = current_idx < target_idx
    df_out.loc[eligible, feature] = target_level
    return df_out, eligible

# Define retrofit actions (single-feature and multi-feature)
actions = {
    "Upgrade lighting to Very Good": {
        "LIGHTING_ENERGY_EFF": "Very Good",
    },
    "Upgrade walls to Good": {
        "WALLS_ENERGY_EFF": "Good",
    },
    "Upgrade windows to Good": {
        "WINDOWS_ENERGY_EFF": "Good",
    },
    "Upgrade main heating to Very Good": {
        "MAINHEAT_ENERGY_EFF": "Very Good",
    },
    "Upgrade hot water to Good": {
        "HOT_WATER_ENERGY_EFF": "Good",
    },
    "Package: fabric + heating": {
        "WALLS_ENERGY_EFF": "Good",
        "WINDOWS_ENERGY_EFF": "Good",
        "MAINHEAT_ENERGY_EFF": "Very Good",
    },
}

list(actions.keys())


['Upgrade lighting to Very Good',
 'Upgrade walls to Good',
 'Upgrade windows to Good',
 'Upgrade main heating to Very Good',
 'Upgrade hot water to Good',
 'Package: fabric + heating']

## Predict counterfactual EPC scores on hold‑out set

For each action, we implement the following steps:
1. Predict baseline EPC score on the hold‑out set.
2. Apply the selected action (only improving eligible dwellings).
3. Predict the counterfactual EPC score.
4. Compute $\Delta \,\text{score} = \text{counterfactual} − \text{baseline}$.

Positive $\Delta$ means the retrofit improves EPC score. Clearly, we anticipate that $\Delta$ will be positive in most cases, although counterintuitive results are plausible.

In [6]:
# Insert your code here

# Guide: 
# - Iterate over actions.items()
# - Find observations (rows) in the test set that are eligible, i.e., for each defined action, 
#    the current band is lower in EFF_ORDER than the target value. For example, for the action "Upgrade lighting to Very Good", 
#    the eligible observations are the ones where test_X["LIGHTING_ENERGY_EFF"] is in ['Very Poor', 'Poor', 'Average', 'Good'].
#    Note that "Package: fabric + heating" contains two actions (eligibility is defined using a logical AND)
# - Predict baseline predictions (without retrofit) and counterfactuals (after applying retrofit)
# - Estimate Delta, provide plots

## Optional: segment impacts by built form

A useful next step is to see whether an action benefits some housing types more than others (e.g., flats vs terraces).


In [7]:
# Insert your code here
